In [9]:
# Basic imports
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import subprocess
from matplotlib import pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import ConcatDataset, DataLoader
import os

In [10]:
class MLPShallow(nn.Module):
    def __init__(self):
        super(MLPShallow, self).__init__()

        self.input_layer = nn.Linear(784, 256)
        self.batch_norm_1 = nn.BatchNorm1d(256)  # BatchNorm po warstwie wejściowej
        self.dropout_1 = nn.Dropout(0.2)  # Dropout z prawdopodobieństwem 20%

        self.hidden_layer_1 = nn.Linear(256, 256)
        self.batch_norm_2 = nn.BatchNorm1d(256)
        self.dropout_2 = nn.Dropout(0.2)

        self.hidden_layer_2 = nn.Linear(256, 128)
        self.batch_norm_3 = nn.BatchNorm1d(128)
        self.dropout_3 = nn.Dropout(0.2)

        self.hidden_layer_3 = nn.Linear(128, 64)
        self.batch_norm_4 = nn.BatchNorm1d(64)
        self.dropout_4 = nn.Dropout(0.2)

        self.hidden_layer_4 = nn.Linear(64, 32)
        self.batch_norm_5 = nn.BatchNorm1d(32)
        self.dropout_5 = nn.Dropout(0.2)

        self.hidden_layer_5 = nn.Linear(32, 16)
        self.batch_norm_6 = nn.BatchNorm1d(16)
        self.dropout_6 = nn.Dropout(0.2)

        self.output_layer = nn.Linear(16, 10)

    def forward(self, x):
        x = F.relu(self.batch_norm_1(self.input_layer(x)))
        x = self.dropout_1(x)

        x = F.relu(self.batch_norm_2(self.hidden_layer_1(x)))
        x = self.dropout_2(x)

        x = F.relu(self.batch_norm_3(self.hidden_layer_2(x)))
        x = self.dropout_3(x)

        x = F.relu(self.batch_norm_4(self.hidden_layer_3(x)))
        x = self.dropout_4(x)

        x = F.relu(self.batch_norm_5(self.hidden_layer_4(x)))
        x = self.dropout_5(x)

        x = F.relu(self.batch_norm_6(self.hidden_layer_5(x)))
        x = self.dropout_6(x)

        x = self.output_layer(x)
        return x

model = MLPShallow()

In [11]:
# please consult torchvision here: https://pytorch.org/vision/stable/index.html
def to_binary(image):
    # Przekształcenie obrazu na wartości binarne: 0 (czarny) i 1 (biały)
    threshold = 0.5  # Ustal próg, gdzie wartości poniżej 0.5 będą czarne, a powyżej białe
    return (image > 0).float()

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Konwersja na czarno-biały obraz
    transforms.ToTensor(),  # Konwersja do tensora
    transforms.Normalize((0.5,), (0.5,)),  # Normalizacja
    transforms.Lambda(to_binary)
])

# first we get the training dataset from touchvision
training_data_mnist = torchvision.datasets.MNIST(root="../data", train=True, transform=transform, download=True)
test_data_mnist = torchvision.datasets.MNIST(root="../data", train=False, transform=transform, download=True)

# EMNIST Dataset (split "digits" dla cyfr 0-9)
training_data_emnist = torchvision.datasets.EMNIST(root="../data", split="digits", train=True, transform=transform, download=True)
test_data_emnist = torchvision.datasets.EMNIST(root="../data", split="digits", train=False, transform=transform, download=True)

# Połączenie wszystkich zbiorów cyfr
full_dataset_train = ConcatDataset([training_data_mnist, training_data_emnist])
full_dataset_test = ConcatDataset([test_data_mnist, test_data_emnist])

# Tworzenie DataLoadera
batch_size = 64
data_loader_train = DataLoader(full_dataset_train, batch_size=batch_size, shuffle=True)
data_loader_test = DataLoader(full_dataset_test, batch_size=batch_size, shuffle=True)

In [12]:
# Create a loss function

# note!! in pytorch the CrossEntropyLoss will apply softmax internally...
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [13]:
Epochs = 3
for epoch in range(Epochs):
  training_loss = 0.0
  correct = 0
  total = 0
  for i, data in enumerate(data_loader_train, 0):
    # get the inputs
    inputs, labels = data
    inputs = inputs.view(inputs.shape[0], -1)
    # Flatten the images

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward
    outputs = model(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    training_loss += loss.item()
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()
    avg_loss = training_loss / (i + 1)
    avg_acc = 100. * correct / total
  print(f'Training Loss: {avg_loss:.3f} | Training acc: {avg_acc:.3f}', 'for epoch: ', epoch)

Training Loss: 0.481 | Training acc: 87.699 for epoch:  0
Training Loss: 0.266 | Training acc: 93.440 for epoch:  1
Training Loss: 0.225 | Training acc: 94.519 for epoch:  2


In [14]:
torch.save(model.state_dict(), 'cnn_6.pth')
print("Model saved to 'cnn.pth'")

Model saved to 'cnn.pth'


In [19]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for i, (image, label) in enumerate(data_loader_test):
      image = image.view(image.shape[0], -1)
      output = model(image)
      test_loss += F.nll_loss(output, label, reduction='sum').item()
      #test_loss += F.nll_loss(output, label, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(label.data.view_as(pred)).sum()
    test_loss /= len(data_loader_test.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(data_loader_test.dataset), 100. * correct / len(data_loader_test.dataset)))


Test set: Avg. loss: -7.0817, Accuracy: 48826/50000 (98%)

